In [3]:
import pandas as pd
import ijson
import glob

In [116]:
time = []
text = []
city = []
location =[]
tweet_id = []
read_files = glob.glob("2020historical/*.json")
for file in read_files:
    with open(str(file), 'r') as f:
        objects = ijson.items(f, 'results.item')
        for obj in objects:
            for row in obj:
                if row['text'].startswith("RT") == False and row['id'] not in location:
                    city.append(row['place']['name'])
                    location.append(row['coordinates'])
                    time.append(row['created_at'])
                    text.append(row['text'])
                    tweet_id.append(row['id'])
data = pd.DataFrame({"text":text,"time":time,"city":city,"location":location},columns=["text","time","city","location"])
data['time'] = pd.to_datetime(data['time']).dt.date

In [117]:
data

,text,time,city,location
0,"@PhillipAdams_1 As my golfer Dad would say, ‘ ...",2020-06-01,Melbourne,None
1,@DavidLeyonhjelm @JoshFrydenberg @political_al...,2020-06-01,Melton,None
2,@in_a_knot @rrogerramjet @SallyRMelb @peacockp...,2020-06-01,Melbourne,None
3,Here is post no. 6 to second part of the What...,2020-06-01,Melbourne,"{'type': 'Point', 'coordinates': [145.06889, -..."
4,@Len_hayes what a legend... next one is indoor...,2020-06-01,Melbourne,None
...,...,...,...,...
14995,@jordanlperkins I think those four words are t...,2020-03-31,Melbourne,None
14996,@JTanganga99 Happy birthday mate! Keep playing...,2020-03-31,Melbourne,None
14997,@mattyperic @DamianTBerry @ziggylisk Chicken S...,2020-03-31,Melbourne,None
14998,"@_DovahGaming_ @Mythical Where's this ""biologi...",2020-03-31,Melbourne,None


In [77]:
from SentimentAnalysis import *

In [78]:
strip_non_ascii_udf = udf(strip_non_ascii, StringType())
fix_abbreviation_udf = udf(fix_abbreviation, StringType())
remove_features_udf = udf(remove_features, StringType())
sentiment_analysis_udf = udf(sentiment_analysis , FloatType())
sentiment_udf = udf(lambda x: condition(x), StringType())

In [79]:
spark = SparkSession.builder.appName('NLP').getOrCreate()

In [80]:
df = spark.createDataFrame(data)
df = df.withColumn('text_non_asci',strip_non_ascii_udf(df['text']))

In [81]:
df = df.withColumn('fixed_abbrev',fix_abbreviation_udf(df['text_non_asci']))

In [82]:
df = df.withColumn('removed',remove_features_udf(df['fixed_abbrev']))

In [83]:
df  = df.withColumn("sentiment_score", sentiment_analysis_udf( df['removed'] ))

In [84]:
df  = df.withColumn("sentiment", sentiment_udf( df['sentiment_score'] ))
SA_results = df.select('text','time','sentiment_score','sentiment')

In [100]:
byMonth = SA_results.select('text','time',month('time').alias('month'),'sentiment_score','sentiment')
SA_resultsMonth = byMonth.groupBy(['month','sentiment']).count().orderBy('month')
# SA_resultsMonth = SA_resultsMonth.filter(SA_resultsMonth.sentiment=="negative").show()
monthly_result = SA_resultsMonth.toPandas()

In [112]:
monthly_result

,month,sentiment,count
0,3,positive,1115
1,3,neutral,1451
2,3,negative,434
3,5,positive,1128
4,5,negative,366
5,5,neutral,1506
6,6,neutral,1460
7,6,negative,371
8,6,positive,1169
9,8,neutral,1437


In [110]:
march = byMonth.filter(byMonth.month==3)
march = march.groupBy(['time','sentiment']).count().orderBy('time')
march_result = march.toPandas()

In [111]:
march_result

,time,sentiment,count
0,2020-03-01,neutral,45
1,2020-03-01,positive,44
2,2020-03-01,negative,11
3,2020-03-02,neutral,59
4,2020-03-02,negative,13
...,...,...,...
85,2020-03-29,neutral,54
86,2020-03-29,negative,16
87,2020-03-31,positive,37
88,2020-03-31,negative,17
